In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
df_train = pd.read_json("./data/train_lem.json")
df_val = pd.read_json("./data/val_lem.json")
df_test = pd.read_json("./data/test_lem.json")

In [3]:
df_train = df_train.sample(frac=1)
df_val = df_val.sample(frac=1)
df_test = df_test.sample(frac=1)

In [4]:
file = open("./data/useless_words.txt")
useless_words = list(file.read().split(" "))

In [5]:
tfidf = TfidfVectorizer(max_features=5000, stop_words=useless_words)

In [6]:
from keras.utils import np_utils

In [7]:
x_train = tfidf.fit_transform(df_train["text"]).toarray()
y_train = np_utils.to_categorical(df_train["subreddit_id"])

x_val = tfidf.transform(df_val["text"]).toarray()
y_val = np_utils.to_categorical(df_val["subreddit_id"])

x_test = tfidf.transform(df_test["text"]).toarray()
y_test = np_utils.to_categorical(df_test["subreddit_id"])

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_dim=5000, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(102),
    tf.keras.layers.Dense(units=102, activation='softmax')
]) 

model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               1280256   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 102)               26214     
_________________________________________________________________
dense_8 (Dense)              (None, 102)               10506     
Total params: 1,316,976
Trainable params: 1,316,976
Non-trainable params: 0
_________________________________________________________________


In [14]:
h = model.fit(x_train, 
              y_train,
              epochs=3,
              batch_size=32,
              validation_data=(x_val, y_val))

Epoch 1/3
1913/1913 [==============================] - 25s 11ms/step - loss: 2.1914 - accuracy: 0.6012 - val_loss: 0.5208 - val_accuracy: 0.8726
Epoch 2/3
1913/1913 [==============================] - 22s 11ms/step - loss: 0.3134 - accuracy: 0.9231 - val_loss: 0.5125 - val_accuracy: 0.8748
Epoch 3/3
1913/1913 [==============================] - 20s 10ms/step - loss: 0.1497 - accuracy: 0.9637 - val_loss: 0.5374 - val_accuracy: 0.8736


In [17]:
results = model.evaluate(x_test, y_test, batch_size=32)
print("test loss, test acc:", results)

638/638 [==============================] - 2s 3ms/step - loss: 0.7190 - accuracy: 0.8411
test loss, test acc: [0.7190294861793518, 0.8410784602165222]


In [12]:
predictions = np.round(model.predict(x_test[:100]))
for i in range(len(predictions)):
    print(df_test["text"].values[i][:100],"...")
    print("Pred: ", predictions[i], "Real: ", y_test[i])

mgur.com/a/iygaf <lb><lb>Hi, I'm pretty ...
Pred:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.] Real:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Best Starting Budget Decks As the title said I just finished the tutorial but I can't win more than  ...
Pred:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.